In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

embedding_dim = 20
tr = .99
counts=[0,0,0,0,0,0,0,0,0]
def convert_to_int(word):
    word_dict = {'Coronavirus':0, 'AskIndia':1, 'Non-Political':2, 'Policy/Economy':3, 'Business/Finance':4, 'Science/Technology':5, 'Sports':6, 'Politics':7,
                'Food':8}
    return word_dict[word]
flair_list=['Coronavirus','AskIndia','Non-Political','Policy/Economy','Business/Finance','Science/Technology','Sports','Politics','Food']
labels=[]
sentences=[]
with open("main_new.csv") as csvfile:
  reader=csv.reader(csvfile,delimiter=',')
  next(reader)
  for row in reader:
      if((row[1] in flair_list) and counts[flair_list.index(row[1])]<2000):
          counts[flair_list.index(row[1])]+=1
          labels.append(convert_to_int(row[1]))
          sentences.append(row[2])

In [0]:
print(len(labels))

In [0]:
from keras.callbacks import ModelCheckpoint
# autosave best Model
best_model = tf.keras.callbacks.ModelCheckpoint('mymodel.h5',monitor='val_acc',verbose=1,save_best_only=True)

In [0]:
x=np.array(labels)
(unique, counts) = np.unique(x, return_counts=True)
print(unique)
print(counts)

In [0]:
import random
c = list(zip(sentences, labels))

random.shuffle(c)

sentences, labels = zip(*c)

In [0]:
tokenizer=Tokenizer(oov_token='<00V>')
tokenizer.fit_on_texts(sentences)
word_index=tokenizer.word_index

seq=tokenizer.texts_to_sequences(sentences)
seq=pad_sequences(seq,padding='post')


training_size=int(len(sentences)*tr)
trainset=seq[:training_size]
trainlabels=labels[:training_size]

In [0]:
testset=seq[training_size:]
testlabels=labels[training_size:]

testlab=np.array((testlabels))
trainlab=np.array((trainlabels))
print(trainlab.shape)
print(trainset.shape)

In [0]:
vocab_size=len(word_index)

In [0]:
from keras import metrics

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim),
    tf.keras.layers.Bidirectional(LSTM(100,return_sequences=False)),
    #tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dense(256,activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(9,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [0]:
history = model.fit(trainset, trainlab, epochs=10, validation_data=(testset, testlab), verbose=2,callbacks=[best_model])

In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")